In [2]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.manifold import TSNE
import random
import warnings
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, f1_score
import community as comm
from community import community_louvain
warnings.filterwarnings('ignore')
%run ./helper.ipynb
#%run ./prediction.ipynb


In [3]:
def node_classification(embeddings, label):
    X, Y = read_node_label(label,skip_head=True)
    
    ltrainfrac = [.7]
    for tf in ltrainfrac:
        print("Training classifier using {:.2f}% nodes...".format(tf * 100))
        split_train_evaluate(X, Y, embeddings, tf)

In [4]:
def makeLinkPredictionData(graph, embeddings):
    # converting embedding to a numpy array
    X = [[0] for i in range(G.number_of_nodes())]
    for i in range(0, G.number_of_nodes()):
        X[i] = embeddings[str(i+1)]
    X = np.array(X)
    
    Xd = []
    Yd = []
    count = 0
    # for all vertices
    for u in range(graph.number_of_nodes()):
        Nu = list(graph.neighbors(u))
        count += len(Nu)
        cn = 0
        totalns = 0
        # for all neighbors of u
        for n in Nu:
            x = []
            if n > u:
                for d in range(len(X[0])):
                    x.append(X[u][d] - X[n][d]) # distance between the embeddings of u and its neighbor n
                Xd.append(x)
                Yd.append(1) # positive sample (edge present)
                totalns += 1
        tmpnn = []
        if len(Nu) > graph.number_of_nodes() // 2:
            totalns = (graph.number_of_nodes() - len(Nu)) // 2
            #print("Testing neighbors!")
        while cn < totalns:
            nn = random.randint(0, graph.number_of_nodes() - 1)
            # non-neighbors of u
            if nn not in Nu and nn not in tmpnn:
                cn += 1
                x = []
                for d in range(len(X[0])):
                    x.append(X[u][d] - X[nn][d])
                Xd.append(x)
                Yd.append(0) # negative sample (edge absent)
                tmpnn.append(nn)
    Xd, Yd = np.array(Xd), np.array(Yd)
    indices = np.array(range(len(Yd)))
    np.random.shuffle(indices)
    Xt = Xd[indices]
    Yt = Yd[indices]
    #print(len(Xd), len(Yd), count)
    
    
    ltrainfrac = [.7]
    for tf in ltrainfrac:
        CV = int(len(Yt) * tf)
        trainX = Xt[0:CV]
        testX = Xt[CV:]
        trainY = Yt[0:CV]
        testY = Yt[CV:]
        modelLR = LogisticRegression().fit(trainX, trainY)
        predictedY = modelLR.predict(testX)
        acc = accuracy_score(predictedY, testY)
        #f1macro = f1_score(predictedY, testY, average='macro', labels=np.unique(predictedY))
        #f1micro = f1_score(predictedY, testY, average='micro', labels=np.unique(predictedY))
        #print("Link predictions:", tf, ":Accuracy:",acc, "F1-macro:", f1macro, "F1-micro:",f1micro)
        print("Link predictions:", tf, ":Accuracy:",acc)

In [5]:
def cluster_eval(G, embeddings):
    # converting embedding to a numpy array
    X = [[0] for i in range(G.number_of_nodes())]
    for i in range(0, G.number_of_nodes()):
        X[i] = embeddings[str(i+1)]
    X = np.array(X)

    bestModularity = 0
    bestC = 10
    NOC = 11
    allmodularity = []
    for cls in range(10, NOC):
        
        # find clusters using a kmeans clustering algorithm on the embedding
        # Number of clusters is set to cls
        clusters = KMeans(n_clusters=cls, random_state=0).fit(X)
        predG = dict()
        for node in range(len(clusters.labels_)):
            predG[node] = clusters.labels_[node]
        
        # compute the modularity score of the Kmeans clustering
        modularity = comm.community_louvain.modularity(predG, G)
        allmodularity.append(modularity)
        print("Number of clusters: ", cls, "  Modularity: ", modularity)
        if modularity > bestModularity:
            bestModularity = modularity
            bestC = cls

In [22]:

def shortcut_walk(G, num_walks, walk_length, p):
    nodes = list(G.nodes())
    walks = []
    for _ in range(num_walks):
        for v in nodes:
            walk = [v]
            while len(walk) < walk_length:
                if random.random() < p:
                    cur = walk[-1]
                    cur_nbrs = list(G.neighbors(cur))
                    if len(cur_nbrs) > 0:
                        walk.append(random.choice(cur_nbrs))
                    else:
                        break
                else:
                    # Select a non-neighbor node
                    cur = walk[-1]
                    nbrs = list(G.neighbors(cur))
                    non_nbrs = [u for u in nodes if u != cur and u not in nbrs]
                    if non_nbrs:
                        walk.append(random.choice(non_nbrs))
                    else:
                        # No non-neighbors available, terminate the walk
                        break
            walks.append(walk)
    return walks


In [18]:
graphfile = 'cora.txt'
labelfile = 'cora.nodes.labels'
G = nx.read_edgelist('cora.txt', nodetype=None)
G = G.to_directed()

In [20]:
walks_deepwalk = shortcut_walk(G, walk_length=10, num_walks=80,p=1)
embeddings_deepwalk001 = get_embedding(G,walks_deepwalk)

Learning embedding vectors...
Learning embedding vectors done!


In [23]:
walks_deepwalk = shortcut_walk(G, walk_length=10, num_walks=80,p=0.5)
embeddings_deepwalk05 = get_embedding(G,walks_deepwalk)

Learning embedding vectors...
Learning embedding vectors done!


In [24]:
walks_deepwalk = shortcut_walk(G, walk_length=10, num_walks=80,p=0.25)
embeddings_deepwalk025 = get_embedding(G,walks_deepwalk)

Learning embedding vectors...
Learning embedding vectors done!


In [26]:
walks_deepwalk = shortcut_walk(G, walk_length=10, num_walks=80,p=0.1)
embeddings_deepwalk01 = get_embedding(G,walks_deepwalk)

Learning embedding vectors...
Learning embedding vectors done!


In [27]:
walks_deepwalk = shortcut_walk(G, walk_length=10, num_walks=80,p=0.01)
embeddings_deepwalk1 = get_embedding(G,walks_deepwalk)

Learning embedding vectors...
Learning embedding vectors done!


<h6>question 3a<h6>

<h6>node classification<h6>

In [29]:
node_classification(embeddings_deepwalk1, labelfile)

Training classifier using 70.00% nodes...
-------------------
{'acc': 0.26535626535626533}
-------------------


In [31]:
node_classification(embeddings_deepwalk01, labelfile)

Training classifier using 70.00% nodes...
-------------------
{'acc': 0.515970515970516}
-------------------


In [32]:
node_classification(embeddings_deepwalk025, labelfile)

Training classifier using 70.00% nodes...
-------------------
{'acc': 0.6339066339066339}
-------------------


In [33]:
node_classification(embeddings_deepwalk05, labelfile)

Training classifier using 70.00% nodes...
-------------------
{'acc': 0.6805896805896806}
-------------------


In [34]:
node_classification(embeddings_deepwalk001, labelfile)

Training classifier using 70.00% nodes...
-------------------
{'acc': 0.7911547911547911}
-------------------


<h6>link prediction<h6>

In [35]:
G1 = G.to_undirected()
G1 = nx.relabel_nodes(G1, lambda x: int(x)-1)

In [36]:
makeLinkPredictionData(G1, embeddings_deepwalk1) 

Link predictions: 0.7 :Accuracy: 0.5115251026207768


In [37]:
makeLinkPredictionData(G1, embeddings_deepwalk01) 

Link predictions: 0.7 :Accuracy: 0.5386801389327439


In [38]:
makeLinkPredictionData(G1, embeddings_deepwalk025) 

Link predictions: 0.7 :Accuracy: 0.5569940006315125


In [39]:
makeLinkPredictionData(G1, embeddings_deepwalk05) 

Link predictions: 0.7 :Accuracy: 0.5648879065361541


In [40]:
makeLinkPredictionData(G1, embeddings_deepwalk001) 

Link predictions: 0.7 :Accuracy: 0.5917271866119356


<h6>clust eval<h6>

In [41]:
cluster_eval(G1, embeddings_deepwalk1)

Number of clusters:  10   Modularity:  0.014506044451426062


In [42]:
cluster_eval(G1, embeddings_deepwalk01)

Number of clusters:  10   Modularity:  0.3566520339886688


In [43]:
cluster_eval(G1, embeddings_deepwalk025)

Number of clusters:  10   Modularity:  0.5926278024806725


In [44]:
cluster_eval(G1, embeddings_deepwalk05)

Number of clusters:  10   Modularity:  0.5118386810429905


In [45]:
cluster_eval(G1, embeddings_deepwalk001)

Number of clusters:  10   Modularity:  0.755320206377621


<h6>Observation: Taking shortcut degrades the quality of trend .Higher value of p gives better result
which represents normal deep walks . P with high values gives best result in all the three cases<h6>